<a href="https://colab.research.google.com/github/ThiPauli/Book-Recommendation-Engine-KNN/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [3]:
#Reading the tab-separated values (TSV) files

train_df = pd.read_table(train_file_path, header=None, names=['type_message', 'message'])
test_df = pd.read_table(test_file_path, header=None, names=['type_message', 'message'])

In [4]:
train_df.head()

,type_message,message
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...


In [5]:
test_df.head()

,type_message,message
0,ham,i am in hospital da. . i will return home in e...
1,ham,"not much, just some textin'. how bout you?"
2,ham,i probably won't eat at all today. i think i'm...
3,ham,don‘t give a flying monkeys wot they think and...
4,ham,who are you seeing?


In [6]:
print(train_df.shape)
print(test_df.shape)

(4179, 2)
(1392, 2)


In [7]:
#Concatenating both df to create the best associations of words and numbers
df_message = pd.concat([train_df, test_df])
print(df_message.shape)

(5571, 2)


In [8]:
#Convert type of message to numeric values.
#Factorize method returns an array of numeric categories and an index of the categories. So index 0 is ham and index 1 is spam.
#It is our labels to train and evaluate the model!

type_message_train = df_message.type_message.factorize()
print(type_message_train)

(array([0, 0, 0, ..., 0, 1, 1]), Index(['ham', 'spam'], dtype='object'))


In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

#Preparing text for NLP
#Using word embeddings, which is capable of capturing the context of a word in a sentence or document.

#Getting all message from the data frame
train_message = df_message.message.values

#Initialize the tokenizer with a 5000 word limit. This is the number of words we would like to encode.
tokenizer = Tokenizer(num_words=5000)

#fit_on_texts to create associations of words and numbers
tokenizer.fit_on_texts(train_message)
print(tokenizer.word_index) #Return a dictionary with key as word and respective value for each word.


vocab_size = len(tokenizer.word_index) + 1

#text_to_sequence replaces the words in a sentence with their respective associated numbers. This transforms each sentence into sequences of numbers.
encoded_message = tokenizer.texts_to_sequences(train_message)

#printing the first message as a text and encoded(integers)
print(train_message[0])
print(encoded_message[0])


#The messages have different number of words, therefore, the length of the sequence of numbers will be different.
print(len(train_message[0]))
print(len(train_message[1]))
#Our model requires inputs to have equal lengths, so we will have to pad the sequence to have the chosen length of inputs.
#All input sequences will have a length of 250.
max_lenght = 250
padded_sequence_message = sequence.pad_sequences(encoded_message, max_lenght)

print(padded_sequence_message[0])
print(len(padded_sequence_message[0]))

{'i': 1, 'to': 2, 'you': 3, 'a': 4, 'the': 5, 'u': 6, 'and': 7, 'in': 8, 'is': 9, 'me': 10, 'my': 11, 'for': 12, 'your': 13, 'it': 14, 'of': 15, 'call': 16, 'have': 17, 'on': 18, '2': 19, 'that': 20, 'now': 21, 'are': 22, 'so': 23, 'but': 24, 'not': 25, 'or': 26, 'do': 27, 'can': 28, 'at': 29, "i'm": 30, 'get': 31, 'will': 32, 'if': 33, 'ur': 34, 'be': 35, 'with': 36, 'just': 37, 'no': 38, 'we': 39, 'this': 40, '4': 41, 'gt': 42, 'lt': 43, 'up': 44, 'when': 45, 'ok': 46, 'from': 47, 'free': 48, 'go': 49, 'how': 50, 'all': 51, 'out': 52, 'what': 53, 'know': 54, 'like': 55, 'good': 56, 'then': 57, 'got': 58, 'come': 59, 'was': 60, 'its': 61, 'am': 62, 'time': 63, 'day': 64, 'only': 65, 'love': 66, 'there': 67, 'send': 68, 'he': 69, 'want': 70, 'text': 71, 'as': 72, 'txt': 73, 'one': 74, 'going': 75, 'by': 76, 'ü': 77, 'need': 78, "i'll": 79, 'home': 80, 'about': 81, 'lor': 82, 'r': 83, 'stop': 84, 'sorry': 85, 'see': 86, 'still': 87, 'n': 88, 'today': 89, 'back': 90, 'da': 91, 'dont': 92

In [10]:
#Build the model
#The model consists of an embedding layer, LSTM layer and a Dense layer which is a fully connected neural network with sigmoid as the activation function.
#Dropouts are added in-between layers and also on the LSTM layer to avoid overfitting.

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.layers import Embedding

embedding_vector_length = 32 # every single word as a vector which has 32 dimensions.

model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length, input_length=max_lenght))

model.add(SpatialDropout1D(0.25))

model.add(LSTM(32, dropout=0.5, recurrent_dropout=0.5))

model.add(Dropout(0.2))

model.add(Dense(1, activation="sigmoid")) #Sigmoid returns values from 0 to 1. Greater than 0.5 means spam, lower than 0.5 means ham!

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 250, 32)           287872    
                                                                 
 spatial_dropout1d (SpatialD  (None, 250, 32)          0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 296,225
Trainable params: 296,225
Non-trainable params: 0
__________________________________________________

In [11]:
#Compile the model
model.compile(loss='binary_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])

In [12]:
#Training the model
#Train data is padded_sequence_message and train label is type_message_train[0] (index 0 because it will get only the array with the numeric values)
#validation_split=0.2 means 20% of the training data to evaluate and validate the model.

history = model.fit(padded_sequence_message, type_message_train[0],
                  validation_split=0.2, epochs=5, batch_size=32)

Epoch 1/5
140/140 [==============================] - 44s 289ms/step - loss: 0.2947 - accuracy: 0.9051 - val_loss: 0.1127 - val_accuracy: 0.9749
Epoch 2/5
140/140 [==============================] - 40s 289ms/step - loss: 0.0682 - accuracy: 0.9818 - val_loss: 0.0595 - val_accuracy: 0.9794
Epoch 3/5
140/140 [==============================] - 40s 288ms/step - loss: 0.0392 - accuracy: 0.9915 - val_loss: 0.0408 - val_accuracy: 0.9883
Epoch 4/5
140/140 [==============================] - 40s 287ms/step - loss: 0.0263 - accuracy: 0.9939 - val_loss: 0.0416 - val_accuracy: 0.9874
Epoch 5/5
140/140 [==============================] - 40s 288ms/step - loss: 0.0188 - accuracy: 0.9951 - val_loss: 0.0358 - val_accuracy: 0.9883


In [13]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  message_encoded = tokenizer.texts_to_sequences([pred_text])
  message_encoded = sequence.pad_sequences(message_encoded, max_lenght)

  prediction_value = model.predict(message_encoded).item()

  #Getting the exact value: 0 or 1 to get either 'ham' or 'spam'.
  prediction_index = int(model.predict(message_encoded).round().item())
  prediction_label = type_message_train[1][prediction_index]

  prediction = [prediction_value, prediction_label]
  return prediction

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

[0.0009475350379943848, 'ham']


In [14]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


You passed the challenge. Great job!
